In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [ ]:
def nba_debuts(year, verbose=False):
    
    """
    Scrapes basketball-reference to obtain player debuts
    """
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_debut.html'.format(year)
    req0 = requests.get(url=url)
    soup = BeautifulSoup(req0.content, 'lxml')

    data_ = soup.find_all('tr')[1:]

    player_data = pd.read_html(str(soup.select('table')), match='Age')[0]
    player_data.columns = player_data.columns.droplevel(0)
    
    playerid = []
    for j in data_:
        if len(j.find_all('a')) > 0:
            pid_v0 = j.find_all('a')[0].get('href')
            pid_v1 = pid_v0.replace('players/', '').replace('.html', '').split('/')[2]
            playerid.append(pid_v1)
    
    player_data = player_data.loc[(player_data['Player'] != 'Player') & 
                                  (player_data['Age'] != 'Shooting')].reset_index(drop=True)
    player_data['debut'] = year
    player_data['playerid'] = playerid
    
    player_data.columns = player_data.columns.str.replace('%', '_pct').str.replace('/', '_').str.\
    replace(' ', '_').str.lower().str.strip()
    
    if verbose:
        keep_col = ['playerid', 'player', 'debut', 'age', 'yrs']
        return player_data.loc[:, keep_col]
    
    return player_data

In [ ]:
A = nba_debuts(year=2010, verbose=True)